# <div align = 'center'> <font color='red'> BOLUM 8
#  <div align = 'center'> <font color='red'>                           XGBOOSTING

In [3]:
import pandas as pd

In [4]:
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('data/LifeExpectancy.csv', index_col=0)

In [17]:
df2 = pd.read_csv('data/Wine.csv')
df2.type =  df2.type.map({'white':0, 'red':1} )

In [18]:
df2.head(1)

,Unnamed: 0,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality_range
0,0,0,7.0,0.27,0.36,17.2,0.045,45.0,170.0,1.001,3.0,0.45,8.8,1


In [19]:
X = df.drop('Life_Expectancy', axis=1)
y = df.Life_Expectancy

In [30]:
X = df2.drop('quality_range', axis=1)
y = df2.quality_range


In [31]:
#Lifed_matrix = xgb.DMatrix(data = X, label=y)

In [32]:
wine_dmatrix = xgb.DMatrix(data = X, label=y)

Modelimizi oluşturuken kullancağımız parametreleri belirliyoruz. XGBoost parametreleri içinde en önemlilerinden bir tanesi objective paramatresidir. Bu parametre ile modelimizin hata fonksiyonu olarak ne kullanacağını belirtiyoruz.

Regresyon problemlerinde reg:linear
Sınıflandırma modellerinde ise, tahmin edilen sınıf için reg-logistic, tahmin ihtimali için ise binary:logistic ifadesi kullanılır.

In [33]:
params = {"objective":"reg:logistic", "max_depth":3, "silent":1}

In [34]:
df2_cv = xgb.cv(dtrain=wine_dmatrix, params=params, nfold=3, 
                    num_boost_round=5, metrics="error", as_pandas=True, seed=123)
display(df2_cv)

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.262660,0.003347,0.271357,0.003454
1,0.256349,0.003424,0.266432,0.006400
2,0.251809,0.004661,0.263199,0.004990
3,0.250501,0.003417,0.264893,0.007354
4,0.244498,0.002306,0.261507,0.007842


xgb.cv fonksiyonunda değerlendirme metriği olarak hata oranı ("error") tanımladık. as_pandas parametresini True olarak belirttiğimiz için sonuç olarak test ve eğitim verileri için hata oranını ve hata oranının standart sapmasını veri çerçevesi olarak döndürdü. Hata değerini 1'den çıkararak doğruluk (accuracy) değerini bulabiliriz.

In [35]:
print('Doğruluk Değeri : {:.3f}'.format((1-df_cv["test-error-mean"]).max()))

Doğruluk Değeri : 0.738


Modelimiz 5 iterasyon sonunda en 0.825 gibi bir duğruluk değerine ulaştı.

Modelimizin performansını AUC skoru ile de değerlendirebiliriz.

In [37]:
wine_auc = xgb.cv(dtrain=wine_dmatrix, params=params, nfold=3, 
                    num_boost_round=5, metrics="auc", as_pandas=True, seed=123)
display(wine_auc)
print('AUC Skoru : {:.3f}'.format(wine_auc["test-auc-mean"].max()))


,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.786421,0.004296,0.768510,0.005279
1,0.803147,0.001637,0.787364,0.005780
2,0.812467,0.002649,0.796505,0.002184
3,0.817142,0.002379,0.800257,0.000868
4,0.822573,0.000661,0.802156,0.003129


AUC Skoru : 0.802


## 2. XGBoost ile Regresyon